# Introduction

`autoplex` is an open-sourced library for automating the generation and benchmarking of machine-learned interatomic potentials (MLIPs). This library provides an easy-to-use interface for fitting different MLIPs. It heavily relies on workflows and jobs implemented in `atomate2`. This library aims to make the development of accurate/reliable MLIPs swift and accessible to a broad community of researchers.

## Currently Supported MLIP architectures

- MACE
- GAP
- NEP
- M3GNET
- J-ACE
- NEQUIP

## Workflows

- Workflow to use random-structure searches for the systematic construction of interatomic potentials: [arXiv: 10.48550/arXiv.2412.16736](https://arxiv.org/abs/2412.16736).
  The implementation automates ideas from the following articles: [*Phys. Rev. Lett.* **120**, 156001 (2018)](https://journals.aps.org/prl/abstract/10.1103/PhysRevLett.120.156001) and [*npj Comput. Mater.* **5**, 99 (2019)](https://www.nature.com/articles/s41524-019-0236-6).
- Workflow to train accurate interatomic potentials for harmonic phonon properties. The implementation automates the ideas from the following article: [*J. Chem. Phys.* **153**, 044104 (2020)](https://pubs.aip.org/aip/jcp/article/153/4/044104/1056348/Combining-phonon-accuracy-with-high).

# Running an Iterative DFT vs MLIP benchmark workflow for phonons

**Info:** autoplex's `CompleteDFTvsMLBenchmarkWorkflow` involves data generation (DFT reference data for forces, stress, and energies with VASP), the fitting of the chosen MLIP, and the benchmarking of the phonons from MLIPs to the reference phonons from DFT calculations.


To systematically converge the quality of the potentials, we recommend using the iterative version of the default `CompleteDFTvsMLBenchmarkWorkflow` i.e. **`IterativeCompleteDFTvsMLBenchmarkWorkflow.`** It will run until the **worst RMSE** value of the benchmark structures falls below a **certain value** or a **maximum number of repetitions** is reached.

One can have a slightly different workflow for the first generation than the subsequent generations. This can help to obtain enough structures for an MLIP fit initially and only slightly increase the number of structures in the next generations. 


<div class="alert alert-block alert-danger"><b>Important:</b> Don’t forget to deactivate the phonon data generation after the first iteration. <b></b></div>

In [ ]:
from atomate2.vasp.flows.core import DoubleRelaxMaker
from atomate2.vasp.jobs.core import StaticMaker, TightRelaxMaker
from atomate2.vasp.jobs.phonons import PhononDisplacementMaker
from atomate2.vasp.sets.core import StaticSetGenerator, TightRelaxSetGenerator
from autoplex.auto.phonons.flows import (
    CompleteDFTvsMLBenchmarkWorkflow,
    IterativeCompleteDFTvsMLBenchmarkWorkflow,
)
from jobflow import Flow
from pymatgen.core.structure import Structure

from jobflow_remote import submit_flow, set_run_config
from atomate2.vasp.powerups import update_user_incar_settings
from atomate2.vasp.powerups import update_vasp_custodian_handlers

We first  initialize the makers for the bulk relax jobs, static DFT runs of single atom displaced structures, and getting isolated atom energies (static DFT run)

In [ ]:
phonon_bulk_relax_maker = DoubleRelaxMaker.from_relax_maker(
    TightRelaxMaker(
        run_vasp_kwargs={"handlers": ()},
        input_set_generator=TightRelaxSetGenerator(
            user_incar_settings={
                "GGA": "PE",
                "ISPIN": 1,
                "KSPACING": 0.2,
                "ALGO": "Fast",
                "LAECHG": False,
                "ISMEAR": 1,
                "ENCUT": 500,
                "IBRION": 1,
                "ISYM": 2,
                "SIGMA": 0.05,
                "LCHARG": False,
                "LWAVE": False,
                "LVTOT": False,
                "LORBIT": None,
                "LOPTICS": False,
                "LREAL": False,
                "ISIF": 4,
                "NPAR": 4,
            }
        ),
    )
)


In [ ]:
phonon_displacement_maker = PhononDisplacementMaker(
    name="dft phonon static",
    run_vasp_kwargs={"handlers": ()},
    input_set_generator=StaticSetGenerator(
        user_incar_settings={
            "GGA": "PE",
            "IBRION": -1,
            "ISPIN": 1,
            "ISMEAR": 1,
            "ISIF": 3,
            "ENCUT": 500,
            "EDIFF": 1e-7,
            "LAECHG": False,
            "LREAL": False,
            "ALGO": "Fast",
            "NSW": 0,
            "LCHARG": False,
            "LWAVE": False,
            "LVTOT": False,
            "LORBIT": None,
            "LOPTICS": False,
            "SIGMA": 0.05,
            "ISYM": 2,
            "KSPACING": 0.2,
            "NPAR": 4,
        },
        auto_ispin=False,
    ),
)

phonon_static_energy_maker = phonon_displacement_maker

In [ ]:
static_isolated_atom_maker = StaticMaker(
    run_vasp_kwargs={"handlers": ()},
    input_set_generator=StaticSetGenerator(
        user_kpoints_settings={"reciprocal_density": 1},
        user_incar_settings={
            "GGA": "PE",
            "ALGO": "Normal",
            "ISPIN": 1,
            "LAECHG": False,
            "ENCUT": 500,
            "ISMEAR": 0,
            "LCHARG": False,
            "LWAVE": False,
            "LVTOT": False,
            "LORBIT": None,
            "LOPTICS": False,
            "NPAR": 4,
        },
    ),
)

Now we collect a number of structures and then optimize them in advance of the workflow. One can also perform subsequent optimizations with different k-point settings, for example.

<div class="alert alert-block alert-info"><b>Note:</b> Due to limited resources and to keep the execution time small, here we use <b>one structure, very small supercells and limit the iterations to 2</b>. Thus, it is expected here that the phonons generated by the fitted MLIP will not be of very high quality and cannot be used for production runs. Ideally you would use much larger supercells and a lot more structures in the workflow<b></b></div>

In [ ]:
job_list = [] # list to collect structure optimization jobs


structure_list = []
benchmark_structure_list = []
start_mpids = ["mp-149"]
start_poscar = ["Si.vasp"]

mpids = []
for mpid, start_poscar in zip(start_mpids, start_poscar):
    for scale in [0.98,1.0]:
        structure = Structure.from_file(start_poscar)
        volume = structure.copy().volume
        structure = structure.scale_lattice((scale**3) * volume)  # added the cube
        job_opt = phonon_bulk_relax_maker.make(structure)
        job_opt.append_name("_" + mpid + "_" + str(scale) + "_pre1")
        job_list.append(job_opt)
        structure_list.append(job_opt.output.structure)
        mpids.append(mpid + "_" + str(scale))


mpbenchmark = mpids
benchmark_structure_list = structure_list

In [ ]:
iteration_flow = IterativeCompleteDFTvsMLBenchmarkWorkflow(
    max_iterations=2,
    rms_max=0.2,
    complete_dft_vs_ml_benchmark_workflow_0=CompleteDFTvsMLBenchmarkWorkflow(
        symprec=1e-3,
        apply_data_preprocessing=True,
        add_dft_rattled_struct=True,
        add_dft_phonon_struct=True,
        volume_custom_scale_factors=[1.0],
        rattle_type=0,
        distort_type=0,
        rattle_std=0.1,  #
        benchmark_kwargs={"relax_maker_kwargs": {"relax_cell": False}},
        supercell_settings={
            "min_length": 3,
            "max_length": 10,
            "min_atoms": 10,
            "max_atoms": 300,
            "fallback_min_length": 9,
        },
        # settings that worked with a GAP
        split_ratio=0.33,
        regularization=False,
        separated=False,
        num_processes_fit=48,
        displacement_maker=phonon_displacement_maker,
        phonon_bulk_relax_maker=phonon_bulk_relax_maker,
        phonon_static_energy_maker=phonon_static_energy_maker,
        rattled_bulk_relax_maker=phonon_bulk_relax_maker,
        isolated_atom_maker=static_isolated_atom_maker,
    ),
    complete_dft_vs_ml_benchmark_workflow_1=CompleteDFTvsMLBenchmarkWorkflow(
        symprec=1e-3,
        apply_data_preprocessing=True,
        add_dft_phonon_struct=False,
        add_dft_rattled_struct=True,
        volume_custom_scale_factors=[1.0],
        rattle_type=0,
        distort_type=0,
        rattle_std=0.1,  # maybe 0.1
        benchmark_kwargs={"relax_maker_kwargs": {"relax_cell": False}},
        supercell_settings={
            "min_length": 3,
            "max_length": 10,
            "min_atoms": 10,
            "max_atoms": 300,
            "fallback_min_length": 9,
        },
        split_ratio=0.33,
        regularization=False,
        separated=False,
        num_processes_fit=48,
        displacement_maker=phonon_displacement_maker,
        phonon_bulk_relax_maker=phonon_bulk_relax_maker,
        phonon_static_energy_maker=phonon_static_energy_maker,
        rattled_bulk_relax_maker=phonon_bulk_relax_maker,
        isolated_atom_maker=static_isolated_atom_maker,
    ),
).make(
    structure_list=structure_list,
    mp_ids=mpids,
    benchmark_structures=benchmark_structure_list,
    benchmark_mp_ids=mpbenchmark,
    rattle_seed=0,
    fit_kwargs_list=[
        {
            "soap": {
                "delta": 1.0,
                "l_max": 12,
                "n_max": 10,
                "atom_sigma": 0.5,
                "zeta": 4,
                "cutoff": 5.0,
                "cutoff_transition_width": 1.0,
                "central_weight": 1.0,
                "n_sparse": 6000,
                "f0": 0.0,
                "covariance_type": "dot_product",
                "sparse_method": "cur_points",
            },
            "general": {
                "two_body": True,
                "three_body": False,
                "soap": True,
                "default_sigma": "{0.001 0.05 0.05 0.0}",
                "sparse_jitter": 1.0e-8,
            },
        }
    ],
)

job_list.append(iteration_flow)
autoplex_flow = Flow(jobs=job_list, output=iteration_flow.output)

In [ ]:
autoplex_flow = update_user_incar_settings(autoplex_flow, {"NPAR": 4})
autoplex_flow = update_vasp_custodian_handlers(autoplex_flow, custom_handlers={})

autoplex_flow.name = "autoplex-simple-phonon-workflow"

In [ ]:
resources = {"nodes": 1, "ntasks":32, "time": "03:00:00"}
submit_flow(autoplex_flow, worker="cecam_autoplex", resources=resources, exec_config="vasp_6.4.3_cecam", project="cecam_school")

In [ ]:
!jf runner start # start the jobflow remote runner if it is shutdown

<div class="alert alert-block alert-warning"><b>Warning:</b> The `write_benchmark_metrics_*` jobs fail with remote error due to a small bug in job connectivity <b></b>. Thus, that job needs to be rerun (This bug is scheduled for a fix with upcoming new release).</div>

#### Access the relax workflow results from the database

In [ ]:
from jobflow_remote import get_jobstore
from autoplex.benchmark.phonons.utils import compare_plot

In [ ]:
# connect to the database where the results are stored
jobstore = get_jobstore()
jobstore.connect()